In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
# %matplotlib inline
import sklearn
from sklearn import datasets, linear_model, model_selection
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score
from sklearn.naive_bayes import GaussianNB
import os
import time
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100
from sklearn.preprocessing import StandardScaler

In [2]:
# Loading in Excel doc into a Dataframe
notebook_path = os.path.abspath("Port-Scan-Naive-Bayes-Normalized.ipynb")
temp = os.path.abspath("Data")
df = pd.read_csv(temp + "\\Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv", low_memory = False)

In [3]:
# Edits the Column Names
df.columns = df.columns.str.replace(' ', '')
df.columns = df.columns.str.replace('/s', '_s')

In [4]:
print(df.Label.value_counts())

PortScan    158930
BENIGN      127537
Name: Label, dtype: int64


In [5]:
# Changes BENIGN and DDoS into 0 and 1 respectively so they can be plotted
df.Label = df.Label.replace('BENIGN', 0)
df.Label = df.Label.replace('PortScan', 1)

In [6]:
# Moves Label to the front of the list
df = df[['Label', 'DestinationPort', 'FlowDuration', 'TotalFwdPackets',
       'TotalBackwardPackets', 'TotalLengthofFwdPackets',
       'TotalLengthofBwdPackets', 'FwdPacketLengthMax', 'FwdPacketLengthMin',
       'FwdPacketLengthMean', 'FwdPacketLengthStd', 'BwdPacketLengthMax',
       'BwdPacketLengthMin', 'BwdPacketLengthMean', 'BwdPacketLengthStd',
       'FlowBytes_s', 'FlowPackets_s', 'FlowIATMean', 'FlowIATStd',
       'FlowIATMax', 'FlowIATMin', 'FwdIATTotal', 'FwdIATMean', 'FwdIATStd',
       'FwdIATMax', 'FwdIATMin', 'BwdIATTotal', 'BwdIATMean', 'BwdIATStd',
       'BwdIATMax', 'BwdIATMin', 'FwdPSHFlags', 'BwdPSHFlags', 'FwdURGFlags',
       'BwdURGFlags', 'FwdHeaderLength', 'BwdHeaderLength', 'FwdPackets_s',
       'BwdPackets_s', 'MinPacketLength', 'MaxPacketLength',
       'PacketLengthMean', 'PacketLengthStd', 'PacketLengthVariance',
       'FINFlagCount', 'SYNFlagCount', 'RSTFlagCount', 'PSHFlagCount',
       'ACKFlagCount', 'URGFlagCount', 'CWEFlagCount', 'ECEFlagCount',
       'Down/UpRatio', 'AveragePacketSize', 'AvgFwdSegmentSize',
       'AvgBwdSegmentSize', 'FwdHeaderLength.1', 'FwdAvgBytes/Bulk',
       'FwdAvgPackets/Bulk', 'FwdAvgBulkRate', 'BwdAvgBytes/Bulk',
       'BwdAvgPackets/Bulk', 'BwdAvgBulkRate', 'SubflowFwdPackets',
       'SubflowFwdBytes', 'SubflowBwdPackets', 'SubflowBwdBytes',
       'Init_Win_bytes_forward', 'Init_Win_bytes_backward', 'act_data_pkt_fwd',
       'min_seg_size_forward', 'ActiveMean', 'ActiveStd', 'ActiveMax',
       'ActiveMin', 'IdleMean', 'IdleStd', 'IdleMax', 'IdleMin']]

In [7]:
df[df.isna().any(axis=1)]

,Label,DestinationPort,FlowDuration,TotalFwdPackets,TotalBackwardPackets,TotalLengthofFwdPackets,TotalLengthofBwdPackets,FwdPacketLengthMax,FwdPacketLengthMin,FwdPacketLengthMean,FwdPacketLengthStd,BwdPacketLengthMax,BwdPacketLengthMin,BwdPacketLengthMean,BwdPacketLengthStd,FlowBytes_s,FlowPackets_s,FlowIATMean,FlowIATStd,FlowIATMax,FlowIATMin,FwdIATTotal,FwdIATMean,FwdIATStd,FwdIATMax,FwdIATMin,BwdIATTotal,BwdIATMean,BwdIATStd,BwdIATMax,BwdIATMin,FwdPSHFlags,BwdPSHFlags,FwdURGFlags,BwdURGFlags,FwdHeaderLength,BwdHeaderLength,FwdPackets_s,BwdPackets_s,MinPacketLength,MaxPacketLength,PacketLengthMean,PacketLengthStd,PacketLengthVariance,FINFlagCount,SYNFlagCount,RSTFlagCount,PSHFlagCount,ACKFlagCount,URGFlagCount,CWEFlagCount,ECEFlagCount,Down/UpRatio,AveragePacketSize,AvgFwdSegmentSize,AvgBwdSegmentSize,FwdHeaderLength.1,FwdAvgBytes/Bulk,FwdAvgPackets/Bulk,FwdAvgBulkRate,BwdAvgBytes/Bulk,BwdAvgPackets/Bulk,BwdAvgBulkRate,SubflowFwdPackets,SubflowFwdBytes,SubflowBwdPackets,SubflowBwdBytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,ActiveMean,ActiveStd,ActiveMax,ActiveMin,IdleMean,IdleStd,IdleMax,IdleMin
12824,0,13370,0,2,0,0,0,0,0,0.0,0.0,0,0,0.0,0.0,NaN,Infinity,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,64,0,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,1,1,0,0,0,0.0,0.0,0.0,64,0,0,0,0,0,0,2,0,0,0,65535,-1,0,32,0.0,0.0,0,0,0.0,0.0,0,0
12827,0,52508,0,1,1,0,0,0,0,0.0,0.0,0,0,0.0,0.0,NaN,Infinity,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,32,32,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,1,0.0,0.0,0.0,32,0,0,0,0,0,0,1,0,1,0,227,65535,0,32,0.0,0.0,0,0,0.0,0.0,0,0
14192,0,46058,0,1,1,0,0,0,0,0.0,0.0,0,0,0.0,0.0,NaN,Infinity,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,32,32,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,1,0.0,0.0,0.0,32,0,0,0,0,0,0,1,0,1,0,227,229,0,32,0.0,0.0,0,0,0.0,0.0,0,0
40707,0,57119,0,2,0,0,0,0,0,0.0,0.0,0,0,0.0,0.0,NaN,Infinity,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,64,0,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,1,1,0,0,0,0.0,0.0,0.0,64,0,0,0,0,0,0,2,0,0,0,65535,-1,0,32,0.0,0.0,0,0,0.0,0.0,0,0
52456,0,16506,0,2,0,0,0,0,0,0.0,0.0,0,0,0.0,0.0,NaN,Infinity,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,64,0,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0.0,0.0,0.0,64,0,0,0,0,0,0,2,0,0,0,229,-1,0,32,0.0,0.0,0,0,0.0,0.0,0,0
56603,0,52995,0,1,1,0,0,0,0,0.0,0.0,0,0,0.0,0.0,NaN,Infinity,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,44,32,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,1,0.0,0.0,0.0,44,0,0,0,0,0,0,1,0,1,0,408,65535,0,44,0.0,0.0,0,0,0.0,0.0,0,0
76549,0,48337,0,1,1,0,0,0,0,0.0,0.0,0,0,0.0,0.0,NaN,Infinity,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,32,32,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,1,0.0,0.0,0.0,32,0,0,0,0,0,0,1,0,1,0,227,229,0,32,0.0,0.0,0,0,0.0,0.0,0,0
76773,0,38790,0,2,0,0,0,0,0,0.0,0.0,0,0,0.0,0.0,NaN,Infinity,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,64,0,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0.0,0.0,0.0,64,0,0,0,0,0,0,2,0,0,0,126,-1,0,32,0.0,0.0,0,0,0.0,0.0,0,0
225846,0,36648,0,1,1,0,0,0,0,0.0,0.0,0,0,0.0,0.0,NaN,Infinity,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,32,32,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,1,0.0,0.0,0.0,32,0,0,0,0,0,0,1,0,1,0,5438,38864,0,32,0.0,0.0,0,0,0.0,0.0,0,0
237968,0,51204,0,2,0,0,0,0,0,0.0,0.0,0,0,0.0,0.0,NaN,Infinity,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,64,0,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0.0,0.0,0.0,64,0,0,0,0,0,0,2,0,0,0,357,-1,0,32,0.0,0.0,0,0,0.0,0.0,0,0


In [8]:
#Drops any rows with NA
df.dropna(how = 'any', subset = ['Label', 'DestinationPort', 'FlowDuration', 'TotalFwdPackets',
       'TotalBackwardPackets', 'TotalLengthofFwdPackets',
       'TotalLengthofBwdPackets', 'FwdPacketLengthMax', 'FwdPacketLengthMin',
       'FwdPacketLengthMean', 'FwdPacketLengthStd', 'BwdPacketLengthMax',
       'BwdPacketLengthMin', 'BwdPacketLengthMean', 'BwdPacketLengthStd',
       'FlowBytes_s', 'FlowPackets_s', 'FlowIATMean', 'FlowIATStd',
       'FlowIATMax', 'FlowIATMin', 'FwdIATTotal', 'FwdIATMean', 'FwdIATStd',
       'FwdIATMax', 'FwdIATMin', 'BwdIATTotal', 'BwdIATMean', 'BwdIATStd',
       'BwdIATMax', 'BwdIATMin', 'FwdPSHFlags', 'BwdPSHFlags', 'FwdURGFlags',
       'BwdURGFlags', 'FwdHeaderLength', 'BwdHeaderLength', 'FwdPackets_s',
       'BwdPackets_s', 'MinPacketLength', 'MaxPacketLength',
       'PacketLengthMean', 'PacketLengthStd', 'PacketLengthVariance',
       'FINFlagCount', 'SYNFlagCount', 'RSTFlagCount', 'PSHFlagCount',
       'ACKFlagCount', 'URGFlagCount', 'CWEFlagCount', 'ECEFlagCount',
       'Down/UpRatio', 'AveragePacketSize', 'AvgFwdSegmentSize',
       'AvgBwdSegmentSize', 'FwdHeaderLength.1', 'FwdAvgBytes/Bulk',
       'FwdAvgPackets/Bulk', 'FwdAvgBulkRate', 'BwdAvgBytes/Bulk',
       'BwdAvgPackets/Bulk', 'BwdAvgBulkRate', 'SubflowFwdPackets',
       'SubflowFwdBytes', 'SubflowBwdPackets', 'SubflowBwdBytes',
       'Init_Win_bytes_forward', 'Init_Win_bytes_backward', 'act_data_pkt_fwd',
       'min_seg_size_forward', 'ActiveMean', 'ActiveStd', 'ActiveMax',
       'ActiveMin', 'IdleMean', 'IdleStd', 'IdleMax', 'IdleMin'], inplace = True)

In [9]:
# Delete rows that contain infinity from FlowBytes and FlowPackets
df = df[~df.FlowBytes_s.str.startswith('Infinity')]
df = df[~df.FlowPackets_s.str.startswith('Infinity')]

In [10]:
# Converts FlowBytes_s and FlowPackets_s to numeric columns
df[['FlowBytes_s']] = df[['FlowBytes_s']].apply(pd.to_numeric)
df[['FlowPackets_s']] = df[['FlowPackets_s']].apply(pd.to_numeric)

In [11]:
df = df[['DestinationPort', 'FlowDuration', 'TotalFwdPackets',
   'TotalBackwardPackets', 'TotalLengthofFwdPackets',
   'TotalLengthofBwdPackets', 'FwdPacketLengthMax', 'FwdPacketLengthMin',
   'FwdPacketLengthMean', 'FwdPacketLengthStd', 'BwdPacketLengthMax',
   'BwdPacketLengthMin', 'BwdPacketLengthMean', 'BwdPacketLengthStd',
   'FlowBytes_s', 'FlowPackets_s', 'FlowIATMean', 'FlowIATStd',
   'FlowIATMax', 'FlowIATMin', 'FwdIATTotal', 'FwdIATMean', 'FwdIATStd',
   'FwdIATMax', 'FwdIATMin', 'BwdIATTotal', 'BwdIATMean', 'BwdIATStd',
   'BwdIATMax', 'BwdIATMin', 'FwdPSHFlags', 'BwdPSHFlags', 'FwdURGFlags',
   'BwdURGFlags', 'FwdHeaderLength', 'BwdHeaderLength', 'FwdPackets_s',
   'BwdPackets_s', 'MinPacketLength', 'MaxPacketLength',
   'PacketLengthMean', 'PacketLengthStd', 'PacketLengthVariance',
   'FINFlagCount', 'SYNFlagCount', 'RSTFlagCount', 'PSHFlagCount',
   'ACKFlagCount', 'URGFlagCount', 'CWEFlagCount', 'ECEFlagCount',
   'Down/UpRatio', 'AveragePacketSize', 'AvgFwdSegmentSize',
   'AvgBwdSegmentSize', 'FwdHeaderLength.1', 'FwdAvgBytes/Bulk',
   'FwdAvgPackets/Bulk', 'FwdAvgBulkRate', 'BwdAvgBytes/Bulk',
   'BwdAvgPackets/Bulk', 'BwdAvgBulkRate', 'SubflowFwdPackets',
   'SubflowFwdBytes', 'SubflowBwdPackets', 'SubflowBwdBytes',
   'Init_Win_bytes_forward', 'Init_Win_bytes_backward', 'act_data_pkt_fwd',
   'min_seg_size_forward', 'ActiveMean', 'ActiveStd', 'ActiveMax',
   'ActiveMin', 'IdleMean', 'IdleStd', 'IdleMax', 'IdleMin', 'Label']]
Y = df.Label
X = df.iloc[:,:-1]
xtrainDF, xtestDF, ytrainDF, ytestDF = train_test_split(X, Y, test_size=0.4, random_state=10)
sc = StandardScaler()
xtrainDF = sc.fit_transform(xtrainDF)
xtestDF = sc.transform(xtestDF)
reg = GaussianNB()
fit = reg.fit(xtrainDF, ytrainDF)
prediction_of_test = fit.predict(xtestDF)
MSE = (np.sqrt(sklearn.metrics.mean_squared_error(ytestDF, prediction_of_test)))
print("MSE: ", MSE)

MSE:  0.554479608153812


In [12]:
# Method that takes model and splites it into test and train data and spits out an MSE
def fit_model(df_data, df_label):
    xtrainDF, xtestDF, ytrainDF, ytestDF = train_test_split(df_data, df_label, test_size=0.4, random_state=10)
    sc = StandardScaler()
    xtrainDF = sc.fit_transform(xtrainDF)
    xtestDF = sc.transform(xtestDF)
    reg = GaussianNB()
    fit = reg.fit(xtrainDF, ytrainDF)
    prediction_of_test = fit.predict(xtestDF)
    return (np.sqrt(sklearn.metrics.mean_squared_error(ytestDF, prediction_of_test)))

# Goes down every column and subcolumn until it peaks at an MSE
def traverse_df(df_data, df_label):
    tmp_mse1, tmp_mse2 = fit_model(df_data, df_label), None
    tmp_df = df_data
    for i in df_data:
        tmp_mse2 = fit_model(df_data.drop(i, axis = 1), df_label)
        if tmp_mse2 < tmp_mse1:
            tmp_mse1 = tmp_mse2
            tmp_df = df_data.drop(i, axis = 1)
    return tmp_mse1, tmp_df



# Main logic for traversing algorithm
df_label = df.Label
df_data = df.iloc[:,:-1]
MSE_list = {}
MSE_list[fit_model(df_data, df_label)] = df_data
j = 0
a = time.time()
for i in df_data:
    print(j)
    j = j + 1
    count = 0
    final_mse = 404
    final_df = None
    MSE, new_df = traverse_df(df_data.drop(i, axis = 1), df_label)
    while True:
        print(new_df.shape)
        if MSE < final_mse:
            print("while: ", count)
            count = count + 1
            final_mse = MSE
            final_df = new_df
        else:
            break
        MSE, new_df = traverse_df(final_df, df_label)
        
    MSE_list[final_mse] = final_df
b = time.time()

0
(286096, 76)
while:  0
(286096, 75)
while:  1
(286096, 74)
while:  2
(286096, 73)
while:  3
(286096, 72)
while:  4
(286096, 72)
1
(286096, 76)
while:  0
(286096, 75)
while:  1
(286096, 74)
while:  2
(286096, 73)
while:  3
(286096, 73)
2
(286096, 76)
while:  0
(286096, 75)
while:  1
(286096, 74)
while:  2
(286096, 73)
while:  3
(286096, 72)
while:  4
(286096, 72)
3
(286096, 76)
while:  0
(286096, 75)
while:  1
(286096, 74)
while:  2
(286096, 73)
while:  3
(286096, 72)
while:  4
(286096, 72)
4
(286096, 76)
while:  0
(286096, 75)
while:  1
(286096, 74)
while:  2
(286096, 73)
while:  3
(286096, 72)
while:  4
(286096, 71)
while:  5
(286096, 70)
while:  6
(286096, 69)
while:  7
(286096, 69)
5
(286096, 76)
while:  0
(286096, 75)
while:  1
(286096, 74)
while:  2
(286096, 73)
while:  3
(286096, 72)
while:  4
(286096, 71)
while:  5
(286096, 71)
6
(286096, 76)
while:  0
(286096, 75)
while:  1
(286096, 74)
while:  2
(286096, 73)
while:  3
(286096, 72)
while:  4
(286096, 71)
while:  5
(286096, 70

(286096, 73)
while:  3
(286096, 72)
while:  4
(286096, 72)
58
(286096, 76)
while:  0
(286096, 75)
while:  1
(286096, 74)
while:  2
(286096, 73)
while:  3
(286096, 72)
while:  4
(286096, 72)
59
(286096, 76)
while:  0
(286096, 75)
while:  1
(286096, 74)
while:  2
(286096, 73)
while:  3
(286096, 72)
while:  4
(286096, 72)
60
(286096, 76)
while:  0
(286096, 75)
while:  1
(286096, 74)
while:  2
(286096, 73)
while:  3
(286096, 72)
while:  4
(286096, 72)
61
(286096, 76)
while:  0
(286096, 75)
while:  1
(286096, 74)
while:  2
(286096, 73)
while:  3
(286096, 72)
while:  4
(286096, 72)
62
(286096, 76)
while:  0
(286096, 75)
while:  1
(286096, 74)
while:  2
(286096, 73)
while:  3
(286096, 72)
while:  4
(286096, 72)
63
(286096, 76)
while:  0
(286096, 75)
while:  1
(286096, 74)
while:  2
(286096, 73)
while:  3
(286096, 72)
while:  4
(286096, 71)
while:  5
(286096, 70)
while:  6
(286096, 69)
while:  7
(286096, 69)
64
(286096, 76)
while:  0
(286096, 75)
while:  1
(286096, 74)
while:  2
(286096, 73)
w

In [13]:
# Takes the list of MSE's and finds the smallest one
fMSE = 404
fdf = None
for key in MSE_list:
    if fMSE > key:
        fMSE = key
        fdf = MSE_list[key]

In [14]:
print(((b-a)/60)/60, " ")

29519.178483486176


In [15]:
fdf

,DestinationPort,TotalFwdPackets,TotalBackwardPackets,FwdPacketLengthMax,FwdPacketLengthMin,FwdPacketLengthStd,BwdPacketLengthMax,FlowIATMin,FwdIATStd,FwdIATMin,FwdPSHFlags,BwdPSHFlags,FwdURGFlags,BwdURGFlags,FwdPackets_s,BwdPackets_s,MinPacketLength,MaxPacketLength,FINFlagCount,SYNFlagCount,RSTFlagCount,PSHFlagCount,ACKFlagCount,URGFlagCount,CWEFlagCount,ECEFlagCount,Down/UpRatio,AvgFwdSegmentSize,FwdAvgBytes/Bulk,FwdAvgPackets/Bulk,FwdAvgBulkRate,BwdAvgBytes/Bulk,BwdAvgPackets/Bulk,BwdAvgBulkRate,SubflowFwdPackets,SubflowBwdPackets,act_data_pkt_fwd
0,22,41,44,456,0,109.864573,976,0,159355.259500,2,0,0,0,0,32.376720,34.745748,0,976,0,0,0,1,0,0,0,0,1,64.975610,0,0,0,0,0,0,41,44,24
1,22,41,44,456,0,109.864573,976,1,159247.900800,1,0,0,0,0,31.075838,33.349680,0,976,0,0,0,1,0,0,0,0,1,64.975610,0,0,0,0,0,0,41,44,24
2,22,1,1,0,0,0.000000,0,160,0.000000,0,0,0,0,0,6250.000000,6250.000000,0,0,0,0,0,0,1,1,0,0,1,0.000000,0,0,0,0,0,0,1,1,0
3,22,41,42,456,0,110.129945,976,0,160397.049900,1,0,0,0,0,31.454068,32.221240,0,976,0,0,0,1,0,0,0,0,1,66.536585,0,0,0,0,0,0,41,42,24
4,35396,1,2,0,0,0.000000,0,28,0.000000,0,0,0,0,0,12987.012990,25974.025970,0,0,0,0,0,0,1,1,0,0,2,0.000000,0,0,0,0,0,0,1,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286462,443,49,57,570,0,108.067176,4344,1,10366.095180,1,0,0,0,0,249.827925,290.616157,0,4344,0,0,0,1,0,0,0,0,1,27.163265,0,0,0,0,0,0,49,57,31
286463,443,49,59,570,0,108.095051,2896,0,34491.462460,2,0,0,0,0,129.484388,155.909773,0,2896,0,0,0,1,0,0,0,0,1,27.040816,0,0,0,0,0,0,49,59,30
286464,443,70,103,570,0,90.746389,4344,1,6109.558409,2,0,0,0,0,432.632880,636.588381,0,4344,0,0,0,1,0,0,0,0,1,20.385714,0,0,0,0,0,0,70,103,47
286465,443,50,62,570,0,107.027727,2896,1,7300.372125,2,0,0,0,0,349.983201,433.979169,0,2896,0,0,0,1,0,0,0,0,1,26.620000,0,0,0,0,0,0,50,62,31


In [16]:
print(fMSE)

0.5177322979303836


In [17]:
fdf.to_csv(os.getcwd() + '//final-Port-Scan-Naive-Bayes-Normalized.csv' , index = False)

In [18]:
for key in MSE_list:
    print(key)

0.554479608153812
0.5486574525315928
0.5484583331702609
0.548362730194635
0.5506288165401926
0.5485459545940573
0.5484822313104114
0.5484424004982988
0.5485539194840265
0.548426467363467
0.5590936812112647
0.5481475625343905
0.5639954438053341
0.5563439656963992
0.5481635037759792
0.5502160512523935
0.5484344339887441
0.5532253151953808
0.5481156786603213
0.5485857778874849
0.5484662993326784
0.549461160116403
0.5480598774174121
0.5177322979303836
0.5484742653793935
